Определение языка (language detection)
--------------------


### Первый метод: частотные слова

#### Скачаем немного википедии для тестов
Воспользуемся пакетом *wikipedia*:

`pip install wikipedia`

In [1]:
from collections import defaultdict
from string import punctuation, digits

Скажу честно, что решила не заморачиваться и взяла тот же список языков, что и был в этом файле изначально.

In [2]:
languages = ['be', 'kk', 'uk', 'fr']

Функция для скачивания статей из википедии:

In [3]:
def get_texts_for_lang(lang, n=10): # функция для скачивания статей из википедии
    wikipedia.set_lang(lang)
    wiki_content = []
    pages = wikipedia.random(n)
    for page_name in pages:
        try:
            page = wikipedia.page(page_name)
        except wikipedia.exceptions.WikipediaException:
            print('Skipping page {}'.format(page_name))
            continue

        wiki_content.append('{}\n{}'.format(page.title, page.content.replace('==', '')))

    return wiki_content

Скачиваем по 100 статей для каждого языка:

In [4]:
import wikipedia # скачиваем по 100 статей для каждого языка. Это может занять какое-то время (5-10 минут. как правило)

wiki_texts = {}
for lang in ('kk', 'uk', 'be', 'fr'): # казахский в википедии — это kk,
                                      # украинский — uk, а белорусский — be
    wiki_texts[lang] = get_texts_for_lang(lang, 100)
    print(lang, len(wiki_texts[lang]))

kk 100


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


Skipping page Беррі
Skipping page Народний депутат (значення)
Skipping page Горпинич
Skipping page Вік (значення)
Skipping page Кам'янобрідська сільська рада
Skipping page Лафлер
Skipping page Теода
uk 93
Skipping page Балдук (значэнні)
Skipping page Банёніс
be 98
Skipping page Sienkiewicz
Skipping page Oustinov
fr 98


Так выглядят тексты:

In [37]:
print(wiki_texts['kk'][0]) # распечатаем пару текстов, чтобы убедиться, что все хорошо
print(wiki_texts['fr'][0])

Вила-Меан (Вила-Нова-ди-Сервейра)
Вила-Меан (Вила-Нова-ди-Сервейра) (порт. Vila Meã (Vila Nova de Cerveira)) — Португалиядағы елді мекен және аудан (фрегезия), Виана-ду-Каштелу округінің құрамына кіреді. Вила-Нова-ди-Сервейра муниципалитетінің құрамдас бөлігі болып табылады.
Португалияның Солтүстік аймақ регионына қарасты Минью-Лима экономика-статистикалық субрегионының құрамына енеді. 2001 жылғы мәліметтер бойынша тұрғындарының саны 269 адамды құрайды. Алып жатқан жер аумағы 2,24 км² шамасында. Халық тығыздығы — 120,1 адам/км².


 Тарихы 
Ескі әкімшілік бөлініс бойынша Минью провинциясының құрамына кірген.


 Сыртқы сілтемелер 
Ресми сайты
Үлгі:Вила-Нова-ди-Сервейра муниципалитетінің аудандары
Dinton (Wiltshire)
Dinton est un village du comté de Wiltshire, Angleterre, Royaume-Uni.
Il offre des commodités dont deux pubs, une boutique assurant le service postal, une église et une salle des fêtes. La rivière Nadder passe au sud du village. Un service d'autocar dessert Salisbury quatre fo

Для создания частотного списка на базе текстов изначально токенизируем текст, чистим и разбиваем на слова:

In [38]:
import codecs
import collections
import re
import sys

def tokenize(text):
    text = re.sub(' - |\. ?|, ?', ' ', text)
    text = re.sub('\d|  +|\(|\)', '', text)
    return text.split(' ')

Словарь, в котором мы будем хранить все частотные списки для каждого из языков
ключ - язык, значение - множество слов

In [39]:
freqs_all = {}

Функция для создания сета частотных слов:

In [40]:
def frequency_set(domain, collection_wiki):
    freqs_lang = collections.defaultdict(lambda: 0)
    corpus = collection_wiki[domain]
    for article in corpus:
        for word in tokenize(article.replace('\n', '').lower()):
            freqs_lang[word] += 1
    return set(freqs_lang.keys())

Создадим такой сет для каждого языка:

In [41]:
kk_set = frequency_set('kk', wiki_texts)
fr_set = frequency_set('fr', wiki_texts)
be_set = frequency_set('be', wiki_texts)
uk_set = frequency_set('uk', wiki_texts)

Исключим повторения из каждого множества (возможно, можно было это сделать и проще либо элегантнее)

In [42]:
uk_set_appr = uk_set - be_set - kk_set - fr_set
be_set_appr = be_set - kk_set - fr_set - uk_set
kk_set_appr = kk_set - be_set - fr_set - uk_set
fr_set_appr = fr_set - be_set - uk_set - kk_set


И добавим получившиеся множества в общий словарь под ключом для каждого конкретного языка:

In [43]:
freqs_all['uk'] = uk_set_appr
freqs_all['be'] = be_set_appr
freqs_all['kk'] = kk_set_appr
freqs_all['fr'] = fr_set_appr

Функция для токенизации заданного текста и сопоставление с частотными списками заданных языков, поиск наиболее сильного пересечения с имеющимся словарем. Возвращает название целевого языка.

In [44]:
def predict_language_freqdic(text, lang_dic_freq, langs):
    text_freqs = tokenize(text.lower())
    lang2sim = {}
    
    for lang in langs:
        intersect = len(set(text_freqs) & lang_dic_freq[lang])
        lang2sim[lang] = intersect

    return max(lang2sim, key=lambda x: lang2sim[x])


### Второй метод: частотные символьные n-граммы

Создадим функцию, которая преобразовывает строку в массив n-грамм заданной длины.

In [46]:
from itertools import islice, tee

def make_ngrams(text):
    text = re.sub(r' - |\. ?|, ?', ' ', text)
    text = re.sub(r'\d|  +|\(|\)|\\|\n|»|«|—| -', '', text)
    N = 3 # задаем длину n-граммы
    l = text.rstrip()
    ngrams = zip(*(islice(seq, index, None) for index, seq in enumerate(tee(l, N))))
    #l = [line.rstrip() for line in ngrams]
    ngrams_l = [''.join(x) for x in ngrams]
    return ngrams_l

Снова - словарь для сета н-грамм для каждого из языков.

In [47]:
ns_freqs_all = {}

Теперь создадим частотные словари n-грамм аналогично первому методу.

In [48]:
def ngrams_set(domain, collection_wiki):
    ngrams_lang = collections.defaultdict(lambda: 0)
    corpus = collection_wiki[domain]
    for article in corpus:
        for word in make_ngrams(article.replace('\n', '').lower()):
            ngrams_lang[word] += 1
    return set(ngrams_lang.keys())

Создаем множество для каждого из языков:

In [49]:
ns_kk_set = ngrams_set('kk', wiki_texts)
ns_fr_set = ngrams_set('fr', wiki_texts)
ns_be_set = ngrams_set('be', wiki_texts)
ns_uk_set = ngrams_set('uk', wiki_texts)

Откидываем пересечения, поскольку они только мешают распознать язык

In [50]:
ns_uk_set_appr = ns_uk_set - ns_be_set - ns_kk_set - ns_fr_set
ns_be_set_appr = ns_be_set - ns_kk_set - ns_fr_set - ns_uk_set
ns_kk_set_appr = ns_kk_set - ns_be_set - ns_fr_set - ns_uk_set
ns_fr_set_appr = ns_fr_set - ns_be_set - ns_uk_set - ns_kk_set


И добавляем каждый из списков н-грамм в общий словарь к конкретному языку

In [51]:
ns_freqs_all['uk'] = ns_uk_set_appr
ns_freqs_all['be'] = ns_be_set_appr
ns_freqs_all['kk'] = ns_kk_set_appr
ns_freqs_all['fr'] = ns_fr_set_appr

Функция определения языка для заданного текста на основе списка н-грамм и списка языков

In [52]:
def predict_language_ngrams(text, ngrams_set, langs):
    text_ngrams = make_ngrams(text.lower())
    lang2sim = {}
    
    for lang in langs:
        intersect = len(set(text_ngrams) & ngrams_set[lang])
        lang2sim[lang] = intersect
    
    return max(lang2sim, key=lambda x: lang2sim[x])


Теперь сравним получающиеся значения на каждом из методов.

Функция принимает на вход соответственно 
dict_texts -- словарь текстов из википедии, где ключ - язык, значение - массив текстов языка
languages - массив названий языков из коллекции текстов
freq_dict_words - множество, наш словарь частотных слов
freq_dict_ngrams - множество частотных n-грамм языка

In [67]:
def test_predictions(dict_texts, languages, freq_dict_words, freq_dict_ngrams):
    ngrams = []
    fre = []
    for lang in languages:
        for el in wiki_texts[lang]:
            f = predict_language_freqdic(el, freqs_all, languages)
            n = predict_language_ngrams(el, ns_freqs_all, languages)
            ngrams.append(n == lang)
            fre.append(f == lang)
    return fre, ngrams

In [68]:
fre, ngrams = test_predictions(wiki_texts, languages, freqs_all, ns_freqs_all)


По текстам, на основе которых был создан словарь, всё довольно очевидно: всё определяется верно, в массивах просто нет значения False.

In [62]:
False in fre

False

In [63]:
False in ngrams

False

Попробуем скачать новый сет текстов и определить язык на основе уже имеющегося частотного словаря.

In [66]:
wiki_texts_test = {}
for lang in languages:
    wiki_texts_test[lang] = get_texts_for_lang(lang, 100)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


Skipping page Мамойкі (Шумілінскі раён)
Skipping page Князеў
Skipping page Варанецкі
Skipping page Падліпкі
Skipping page Тошчыца (значэнні)
Skipping page Філмар
Skipping page Көксу көшесі (Алматы)
Skipping page Українізація (значення)
Skipping page Трофа (значення)
Skipping page 1547 (значення)
Skipping page Волянський
Skipping page Вишнівський
Skipping page Каріно
Skipping page Zdrój
Skipping page Billy Gilbert
Skipping page Le Dernier Souffle (homonymie)
Skipping page Zakrzewo Wielkie
Skipping page TMAP
Skipping page Kambatta


In [ ]:
def test_predictions(dict_texts, languages, freq_dict_words, freq_dict_ngrams):
    ngrams = []
    fre = []
    for lang in languages:
        for el in wiki_texts[lang]:
            f = predict_language_freqdic(el, freqs_all, languages)
            n = predict_language_ngrams(el, ns_freqs_all, languages)
            ngrams.append(n == lang)
            fre.append(f == lang)
    return fre, ngrams

In [69]:
freq_test, ngrams_test = test_predictions(wiki_texts_test, languages, freqs_all, ns_freqs_all)


In [71]:
False in freq_test

False

In [72]:
False in ngrams_test

False

Снова всё ок.